# USD JPY Strategy by Wolfrank Guzman


## Overview
This project implements a backtest for a USD/JPY forex trading strategy using Python. The strategy involves two main components:
1. Selling after three consecutive red (negative) 15-minute candles
2. Buying at the daily pivot point

## Strategy Details
- Sell Entry: Close of the last 15-minute candle if the previous three candles were red (negative)
- Buy Entry: Bounce off the daily pivot point
- Stop Loss: 15 pips
- Take Profit: 25 pips

## Dependencies
- pandas
- yfinance
- quantstats

## Installation
pip install pandas yfinance quantstats


## Usage
Run the `usd_jpy_backtest.py` script to execute the backtest:
python usd_jpy_backtest.py

## Output
The script generates the following outputs:
1. A CSV file with the backtest results: `usd_jpy_backtest_results.csv`
2. QuantStats HTML report: `usd_jpy_backtest_report.html`

Both files will be saved in the same directory as the script.

## Notes
- The backtest uses historical data from Yahoo Finance
- Adjust the date range in the script to test different periods
- Consider transaction costs and slippage for more accurate results in a real trading environment






@guzmanwolfrank : Github



In [1]:
import pandas as pd
import yfinance as yf
import quantstats as qs
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')



# Download USD/JPY data
symbol = "USDJPY=X"
start_date = "2024-06-08"
end_date = "2024-07-08"

data = yf.download(symbol, start=start_date, end=end_date, interval="15m")

# Calculate daily pivot point
data['Pivot'] = (data['High'].resample('D').last() + data['Low'].resample('D').last() + data['Close'].resample('D').last()) / 3
data['Pivot'] = data['Pivot'].fillna(method='ffill')

# Initialize columns for signals and positions
data['Signal'] = 0
data['Position'] = 0

# Implement sell strategy
for i in range(3, len(data)):
    if (data['Close'].iloc[i-1] < data['Open'].iloc[i-1] and 
        data['Close'].iloc[i-2] < data['Open'].iloc[i-2] and 
        data['Close'].iloc[i-3] < data['Open'].iloc[i-3]):
        data['Signal'].iloc[i] = -1

# Implement buy strategy
data['Signal'] = np.where((data['Low'] <= data['Pivot']) & (data['Close'] > data['Pivot']), 1, data['Signal'])

# Set stop loss and take profit levels
stop_loss_pips = 0.15
take_profit_pips = 0.25

# Calculate returns
data['Returns'] = 0.0

for i in range(1, len(data)):
    if data['Signal'].iloc[i-1] != 0:
        entry_price = data['Close'].iloc[i-1]
        stop_loss = entry_price - stop_loss_pips if data['Signal'].iloc[i-1] == 1 else entry_price + stop_loss_pips
        take_profit = entry_price + take_profit_pips if data['Signal'].iloc[i-1] == 1 else entry_price - take_profit_pips
        
        j = i
        while j < len(data):
            if data['Signal'].iloc[i-1] == 1:  # Long position
                if data['Low'].iloc[j] <= stop_loss:
                    data['Returns'].iloc[j] = (stop_loss - entry_price) / entry_price
                    break
                elif data['High'].iloc[j] >= take_profit:
                    data['Returns'].iloc[j] = (take_profit - entry_price) / entry_price
                    break
            else:  # Short position
                if data['High'].iloc[j] >= stop_loss:
                    data['Returns'].iloc[j] = (entry_price - stop_loss) / entry_price
                    break
                elif data['Low'].iloc[j] <= take_profit:
                    data['Returns'].iloc[j] = (entry_price - take_profit) / entry_price
                    break
            j += 1

# Calculate cumulative returns
data['Cumulative_Returns'] = (1 + data['Returns']).cumprod()

# Save results to CSV
csv_path = os.path.join(os.getcwd(), 'usd_jpy_backtest_results.csv')
data.to_csv(csv_path)
print(f"Backtest results saved to: {csv_path}")

# Generate QuantStats report
qs_returns = data['Returns'].copy()
qs_returns.index = pd.to_datetime(qs_returns.index)
qs_returns = qs_returns[qs_returns != 0].dropna()

report_path = os.path.join(os.getcwd(), 'usd_jpy_backtest_report.html')
qs.reports.html(qs_returns, output=report_path, title="USD/JPY Trading Strategy Backtest")
print(f"QuantStats report saved to: {report_path}")

# Print summary statistics
print("\nBacktest Summary:")
print(f"Total Trades: {len(qs_returns)}")
print(f"Cumulative Return: {data['Cumulative_Returns'].iloc[-1]:.2%}")
print(f"Sharpe Ratio: {qs.stats.sharpe(qs_returns):.2f}")
print(f"Max Drawdown: {qs.stats.max_drawdown(qs_returns):.2%}")

[*********************100%%**********************]  1 of 1 completed


Backtest results saved to: c:\Users\Wolfrank\Desktop\CodeWolf\QuantTrading\Algorithmic Backtests\QuantTrading\Algorithmic Backtests\USDJPY\usd_jpy_backtest_results.csv
QuantStats report saved to: c:\Users\Wolfrank\Desktop\CodeWolf\QuantTrading\Algorithmic Backtests\QuantTrading\Algorithmic Backtests\USDJPY\usd_jpy_backtest_report.html

Backtest Summary:
Total Trades: 154
Cumulative Return: 95.36%
Sharpe Ratio: -4.45
Max Drawdown: -4.93%
